## 模型参数的访问、初始化和共享
---

### 访问模型参数

In [15]:
import torch
from torch import nn
from torch.nn import init

net = nn.Sequential(nn.Linear(4, 3), nn.ReLU(), nn.Linear(3,1))

print(net)
X = torch.rand(2, 4)
Y = net(X).sum()

Sequential(
  (0): Linear(in_features=4, out_features=3, bias=True)
  (1): ReLU()
  (2): Linear(in_features=3, out_features=1, bias=True)
)


In [16]:
print(type(net.named_parameters()))

for name, param in net.named_parameters():
    print(name, param.size())

<class 'generator'>
0.weight torch.Size([3, 4])
0.bias torch.Size([3])
2.weight torch.Size([1, 3])
2.bias torch.Size([1])


In [17]:
# 通过方括号[]来访问网络的任一层
# 索引0表示隐藏层为Sequential实例最先添加的层
for name,param in net[0].named_parameters():
    print(name, param.size(), type(param))

weight torch.Size([3, 4]) <class 'torch.nn.parameter.Parameter'>
bias torch.Size([3]) <class 'torch.nn.parameter.Parameter'>


In [18]:
class MyModel(nn.Module):
    def __init__(self, **kwargs):
        super(MyModel, self).__init__(**kwargs)
        self.weight1 = nn.Parameter(torch.rand(20, 20))
        self.weight2 = torch.rand(20, 20)
n = MyModel()
for name, param in n.named_parameters():
    print(name)

weight1


In [19]:
weight_0 = list(net[0].parameters())[0]
print(weight_0.data)
print(weight_0.grad)
Y.backward()
print(weight_0.grad)

tensor([[-0.0916,  0.2473,  0.3371, -0.0763],
        [ 0.3191, -0.0727, -0.4604,  0.4694],
        [-0.4827, -0.1430, -0.2973, -0.4631]])
None
tensor([[-0.1916, -0.0256, -0.0897, -0.2807],
        [ 0.1096,  0.0147,  0.0513,  0.1606],
        [ 0.0000,  0.0000,  0.0000,  0.0000]])


### 初始化模型参数

In [20]:
for name, param in net.named_parameters():
    if 'weight' in name:
        init.normal_(param, mean=0, std=0.01)
        print(name, param.data)

0.weight tensor([[-0.0041,  0.0038,  0.0103, -0.0124],
        [ 0.0131,  0.0018, -0.0048,  0.0037],
        [ 0.0002, -0.0040,  0.0020,  0.0059]])
2.weight tensor([[ 0.0056,  0.0074, -0.0062]])


In [21]:
# 使用常数来初始化权重参数
for name, param in net.named_parameters():
    if 'bias' in name:
        init.constant_(param, val=0)
        print(name, param.data)

0.bias tensor([0., 0., 0.])
2.bias tensor([0.])


### 自定义初始化方法

In [22]:
def normal_(tensor, mean=0, std=1):
    with torch.no_grad():
        return tensor.normal_(mean, std)

In [23]:
def init_weight_(tensor):
    with torch.no_grad():
        tensor.uniform_(-10, 10)
        tensor *= (tensor.abs() >= 5).float() # 保留[−10,−5][−10,−5]和[5,10][5,10]两个区间里均匀分布的随机数

for name, param in net.named_parameters():
    if 'weight' in name:
        init_weight_(param)
        print(name, param.data)

0.weight tensor([[-0.0000,  0.0000, -0.0000,  5.0871],
        [ 5.7874, -7.2956,  6.5675, -9.8095],
        [-0.0000, -5.6092,  0.0000, -5.6658]])
2.weight tensor([[-0.0000, -0.0000, 8.9397]])


In [24]:
for name, param in net.named_parameters():
    if 'bias' in name:
        param.data += 1
        print(name, param.data)

0.bias tensor([1., 1., 1.])
2.bias tensor([1.])


### 共享模型参数
Module类的forward函数里多次调用同一个层。此外，如果我们传入Sequential的模块是同一个Module实例的话参数也是共享的

In [25]:
linear = nn.Linear(1, 1, bias=False)
net = nn.Sequential(linear, linear)
print(net)
for name, param in net.named_parameters():
    init.constant_(param, val=3)
    print(name, param.data)

Sequential(
  (0): Linear(in_features=1, out_features=1, bias=False)
  (1): Linear(in_features=1, out_features=1, bias=False)
)
0.weight tensor([[3.]])


In [26]:
# 在内存中，这两个线性层其实一个对象
print(id(net[0]) == id(net[1]))
print(id(net[0].weight) == id(net[1].weight))

True
True


In [27]:
x = torch.ones(1, 1)
y = net(x).sum()
print(y)
y.backward()
print(net[0].weight.grad)
y# 单次梯度是3，两次所以就是6

tensor(9., grad_fn=<SumBackward0>)
tensor([[6.]])


tensor(9., grad_fn=<SumBackward0>)

In [28]:
l = torch.ones(2, 2)
l

tensor([[1., 1.],
        [1., 1.]])

tensor([[9.]], grad_fn=<MmBackward>)